In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from itertools import cycle
from mpl_toolkits.mplot3d import Axes3D
from sklearn import cluster,mixture
from sklearn.datasets import make_blobs
from sklearn.neighbors import kneighbors_graph
%matplotlib inline

In [3]:
data = pd.read_csv('Rat_Sightings.csv')
#data = data.iloc[:, 15:26]
#data.head(15)

,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Action Updated Date,Community Board,Borough,X Coordinate (State Plane),Y Coordinate (State Plane)
0,INTERSECTION,NEW YORK,NaN,NaN,Closed,10/04/2015 03:01:02 PM,09/18/2015 12:00:00 AM,01 MANHATTAN,MANHATTAN,980656.0,197137.0
1,LATLONG,STATEN ISLAND,NaN,NaN,Closed,10/04/2015 10:02:58 AM,10/28/2015 12:00:00 AM,Unspecified STATEN ISLAND,STATEN ISLAND,955207.0,148858.0
2,ADDRESS,STATEN ISLAND,NaN,NaN,Assigned,10/04/2015 11:58:02 AM,09/04/2015 12:03:49 PM,01 STATEN ISLAND,STATEN ISLAND,949033.0,169278.0
3,ADDRESS,BROOKLYN,NaN,NaN,Closed,10/04/2015 06:56:34 AM,09/14/2015 12:00:00 AM,01 BROOKLYN,BROOKLYN,1000550.0,197585.0
4,ADDRESS,BRONX,NaN,NaN,Closed,10/04/2015 10:08:27 AM,09/22/2015 12:00:00 AM,11 BRONX,BRONX,1021648.0,250489.0
5,ADDRESS,BROOKLYN,NaN,NaN,Closed,10/04/2015 07:59:34 PM,09/22/2015 12:00:00 AM,06 BROOKLYN,BROOKLYN,984607.0,186007.0
6,ADDRESS,Flushing,NaN,NaN,Closed,10/04/2015 11:42:10 AM,09/22/2015 04:26:36 PM,07 QUEENS,QUEENS,1032971.0,218993.0
7,ADDRESS,BROOKLYN,NaN,NaN,Assigned,10/04/2015 11:55:26 AM,09/04/2015 11:59:22 AM,03 BROOKLYN,BROOKLYN,994926.0,192806.0
8,INTERSECTION,NEW YORK,NaN,NaN,Pending,10/04/2015 10:48:16 AM,09/04/2015 10:51:20 AM,05 MANHATTAN,MANHATTAN,988029.0,208946.0
9,ADDRESS,STATEN ISLAND,NaN,NaN,Closed,10/04/2015 07:51:16 AM,09/25/2015 12:00:00 AM,01 STATEN ISLAND,STATEN ISLAND,939187.0,171503.0


In [ ]:
data = data.sample(frac=0.1, random_state=200)
#data.shape
#test_data  = data.drop(train_data.index)
#print(train_data.shape, test_data.shape)

In [ ]:
data = data.iloc[:, 24:26]

Esse é um jeito de plotar

In [ ]:
#pega os dados do data set
list1=list(data['X Coordinate (State Plane)'])
list2=list(data['Y Coordinate (State Plane)'])

#transforma os dados em uma lista conjunta
X = zip(list1,list2)
dt=np.dtype('int')
listaSemNaN=[]
X=list(X)

#remove dessa lista valores que causam erros por não serem numeros
for i in range(0,len(X)):
    tup=X[i]
    if((X[i][0] == X[i][0]) is False or (X[i][1] == X[i][1]) is False):
        nada = 0 #print "Não é NaN"
    else:
        listaSemNaN.append(tup)
        
#Transforma em um NP.Array para realizar contas matematicas da blibioteca numpy
X = np.array(listaSemNaN,dtype=dt)


In [ ]:
num_cluster=3
neighbors=10

bandwidth=cluster.estimate_bandwidth(X,n_samples=1000)

# connectivity matrix for structured Ward
connectivity = kneighbors_graph(X,n_neighbors=neighbors, include_self=False)
# make connectivity symmetric
connectivity = 0.5 * (connectivity + connectivity.T)

meanshift = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
two_means = cluster.MiniBatchKMeans(n_clusters=num_cluster)
ward = cluster.AgglomerativeClustering(n_clusters=num_cluster, linkage='ward',connectivity=connectivity)
spectral = cluster.SpectralClustering(n_clusters=num_cluster, eigen_solver='arpack',affinity="nearest_neighbors")
avg_linkage = cluster.AgglomerativeClustering(linkage="average", affinity="cityblock",n_clusters=num_cluster,connectivity=connectivity)
birch = cluster.Birch(n_clusters=num_cluster)
gmm = mixture.GaussianMixture(n_components=num_cluster, covariance_type='full')
dbscan = cluster.DBSCAN()
affinity_propagation = cluster.AffinityPropagation()


In [ ]:
meanshift = meanshift.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=meanshift)

In [ ]:
two_means = two_means.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=two_means)

In [ ]:
#sem memoria suficiente Memory Error
ward= ward.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=ward)

In [ ]:
#Warning sobre conectividade
#spectral = spectral.fit_predict(X)
#plt.scatter(X[:, 0], X[:, 1], c=spectral)

In [ ]:
#Memory Error
avg_linkage = avg_linkage.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=avg_linkage)

In [ ]:
#Memory Error
birch = birch.fit_predict(X)
plt.scatter(X[:,0],X[:,1],c=birch)

In [ ]:
gmm = gmm.fit(X)
gmm = gmm.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=gmm)

In [ ]:
#Memory Error
affinity_propagation = affinity_propagation.fit_predict(X)
plt.scatter(X[:,0],X[:,1],c=affinity_propagation)

In [ ]:
dbscan = dbscan.fit_predict(X)
plt.scatter(X[:,0],X[:,1],c=dbscan)